- DTM(Document-Term Matrix)
    - 문서 단어행렬 표현방법: 서로 다른 문서들의 BoW들을 결합한 표현방법
- TF-IDF(Term Frequency-Inverse Document Frequency)
    - 문서의 빈도에 특정 식을 취하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주어 계산하는 방법이다.
- TF-IDF가 왜 필요할까?
    - 각 문서에는 중요한 단어와 불필요한 단어들이 혼재되어 있다. 자주 등장하는 단어여도 즉, 빈도수가 높더라도 해당 문서에서 중요한 역할을 하는 단어가 아닐 가능성도 있기 때문에 다른 문서에서 빈도수가 낮고 해당 문서에서 빈도수가 높다면 해당 문서에서 중요한 단어일 가능성이 크다.
- TF-IDF는 어디에 활용될까?
    - 문서의 유사도를 구하는 작업
    - 검색 시스템에서 검색 결과의 중요도를 정하는 작업
    - 문서 내에서 특정 단어의 중요도를 구하는 작업


TF-IDF 계산 수식

- 문서: d, 단어: t, 문서의 총 개수: n
- tf(d,t): 특정 문서 d에서의 특정 단어 t의 등장 횟수
- df(t): 특정 단어 t가 등장한 문서의 수
- idf(d,t): df(t)에 반비례하는 수, 여러 문서에서 등장한 단어의 가중치를 낮추는 역할

![idf 수식](./idf.png)

질문
- 왜 log를 취할까?
    - df의 역수에 로그를 취하지 않으면, 총 문서의 수인 n이 커질수록 idf 값은 기하급수적으로 커지게 되기 때문이다.
    - 불용어 등과 같이 자주 쓰이는 단어들은 자주 쓰이지 않는 단어들보다 최소 수십배 자주 등장하는데, 이 경우, 로그를 씌워주지 않으면 불용어에 엄청난 가중치가 부여될 수 있다. 
- 왜 분모에 1을 더할까?
    - 특정 단어가 전체 문서에서 등장하지 않을 경우에 분모가 0이 되는 상황을 방지하기 위함이다.

In [6]:
import pandas as pd
from math import log

docs = [
'먹고 싶은 사과',
'먹고 싶은 바나나',
'길고 노란 바나나 바나나',
'저는 과일이 좋아요'
]

#모든 문서의 모든 단어에 대해서 중복을 없애고 리스트로 변환
vocab = list(set(w for doc in docs for w in doc.split()))
#가나다 순으로 정렬
vocab.sort()


['과일이', '바나나', '노란', '사과', '싶은', '저는', '먹고', '좋아요', '길고']
['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']


In [22]:
#총 문서의 수
N = len(docs)
#TF
def tf(t,d):
    return d.count(t)
#IDF
def idf(t):
    df = 0
    for doc in docs:
        #vocab에 있는 각 단어가 몇 개의 문서에 존재하는지 확인
        #true or false를 + 연산하면 0 or 1이 더해짐
        df += t in doc
    return log(N/(df+1))
#TF-IDF
def tfidf(t,d):
    return tf(t,d) * idf(t)

In [20]:
var = 0
var += True
print(var)

1


In [13]:
#TF구하고 DF로 저장하여 출력

result = []
for i in range(N):
    #각 문서마다 다른 행에 vocab 빈도수 리스트가 저장되게 하기위함
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        #각 문서마다 vocab에 있는 단어들의 빈도수를 카운트하여 result 리스트의 마지막 요소에 append 
        result[-1].append(tf(t,d))
        
tf_ = pd.DataFrame(result, columns=vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [23]:
#IDF 구해서 DF로 저장하여 출력

result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))
    
idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [24]:
result = []

for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns=vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


- 위에서 구현했던 TF-IDF 식의 문제점
    - 만약 전체 문서수가 4인데, df의 값이 3인 경우는?
        - df+1이 되어서 로그항의 분자와 분모가 값이 같아진다. 즉, idf값이 0이 되어 가중치 역할을 수행할 수 없게 된다.

사이킷 런의 TF-IDF는 위의 식에서 조정된 식을 사용한다.

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',
]

vector = CountVectorizer()

#코퍼스로부터 각 단어의 빈도수 리스트를 저장
print(vector.fit_transform(corpus).toarray())

#각 단어와 매핑된 인덱스 출력
print(vector.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',
]

tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
